In [1]:
!pip install pytesseract opencv-python pillow
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [9]:
import cv2
import pytesseract
from PIL import Image

# If Tesseract is not in your PATH, specify its location:
# pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Load image via OpenCV and convert to RGB (pytesseract expects RGB)
image = cv2.imread('12.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# (Optional) Preprocessing: convert to grayscale, thresholding to improve accuracy
gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
_, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

# Extract text
text = pytesseract.image_to_string(thresh, lang='eng')
print("Detected Text:")
print(text)


Detected Text:
[i 1 WELeenie REPUBLIC
log -

 

Questions & Answers

This document provides answers to frequently asked questions. It is updated on a regular basis

Pre-Registration, Full Registration, Transportation, Procedural issues.

General information

The Appointments are scheduled for everyone who has pre-registered for asylum. All persons
who have pre-registered will be summaned ta an appointment at the Asylum Service, depending
‘on piace of residence at the moment of pre-registration, as well as type of procedure (explained
further). This appointment isthe next step for your application far asylum in Greece, for family
Reunification or for relacation elsewhere in Eurape.

The schedule for the appointments is available at the website of the Greek Asylum Service —
wvew.asylo gov.er. The printed version is available alsa at all the officiat sites and camps. Please
appraach UNHCR staff to consul it. SMS were sent for September anly,far the other manths SMS
will be sent forthe app

In [5]:
ngrok.set_auth_token("set_your_token_ ngrok")


In [8]:
# Complete script to run in Google Colab
# ------------------------------------------

# STEP 1: Install necessary packages
!pip install fastapi uvicorn pyngrok pytesseract opencv-python Pillow pydantic python-multipart nest_asyncio

# STEP 2: Install Tesseract OCR
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-eng

# STEP 3: Create and run the FastAPI application
# Save this cell and run it last

import uvicorn
from fastapi import FastAPI, HTTPException, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import cv2
import numpy as np
import base64
import pytesseract
import logging
from pyngrok import ngrok
import nest_asyncio

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize FastAPI app
app = FastAPI(title="Receipt OCR API")

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # For production, restrict to your domain
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Define request model for base64 images
class Base64ImageRequest(BaseModel):
    image: str

# Preprocess image for better OCR results
def preprocess_image(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding - works better for receipts with varying lighting
    binary = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    )

    # Apply noise reduction
    denoised = cv2.fastNlMeansDenoising(binary, None, 10, 7, 21)

    # Optional: Dilation to make text more prominent
    kernel = np.ones((1, 1), np.uint8)
    dilated = cv2.dilate(denoised, kernel, iterations=1)

    return dilated

@app.get("/")
def read_root():
    return {"message": "Receipt OCR API is running!"}

@app.post("/ocr/base64/")
async def ocr_base64(request: Base64ImageRequest):
    try:
        logger.info("Processing base64 image")

        # Strip out the base64 encoding header if present
        if "," in request.image:
            base64_data = request.image.split(",")[1]
        else:
            base64_data = request.image

        # Decode base64 image
        image_bytes = base64.b64decode(base64_data)
        image_array = np.frombuffer(image_bytes, np.uint8)

        # Decode image
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
        if image is None:
            raise HTTPException(status_code=400, detail="Could not decode image")

        # Preprocess image
        preprocessed = preprocess_image(image)

        # Save preprocessed image for debugging (optional)
        # cv2.imwrite("preprocessed.jpg", preprocessed)

        # Perform OCR with custom configuration for better receipt handling
        custom_config = r'--oem 3 --psm 6'  # Page segmentation mode 6: Assume single uniform block of text
        text = pytesseract.image_to_string(preprocessed, config=custom_config)

        # Log success
        logger.info(f"OCR completed. Text length: {len(text)}")

        return {"text": text, "status": "success"}

    except Exception as e:
        logger.error(f"Error processing image: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Error processing image: {str(e)}")

@app.post("/ocr/upload/")
async def ocr_upload(file: UploadFile = File(...)):
    try:
        # Read uploaded file
        contents = await file.read()
        image_array = np.frombuffer(contents, np.uint8)

        # Decode image
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
        if image is None:
            raise HTTPException(status_code=400, detail="Could not decode image")

        # Preprocess image
        preprocessed = preprocess_image(image)

        # Perform OCR with custom configuration
        custom_config = r'--oem 3 --psm 6'
        text = pytesseract.image_to_string(preprocessed, config=custom_config)

        return {"text": text, "status": "success"}

    except Exception as e:
        logger.error(f"Error processing uploaded file: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Error processing uploaded file: {str(e)}")

# STEP 4: Set up ngrok and start the server
# Uncomment and replace with your auth token if you have one
# ngrok.set_auth_token("YOUR_AUTHTOKEN_HERE")

# Apply nest_asyncio to allow running async code in notebook
nest_asyncio.apply()

# Start ngrok tunnel
ngrok_tunnel = ngrok.connect(8000)
print(f"✅ Public URL: {ngrok_tunnel.public_url}")
print(f"👉 Update this URL in your Flutter app's settings")
print(f"📝 API Documentation: {ngrok_tunnel.public_url}/docs")

# Start the FastAPI application
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

INFO:     Started server process [6335]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


✅ Public URL: https://1058-34-16-136-168.ngrok-free.app
👉 Update this URL in your Flutter app's settings
📝 API Documentation: https://1058-34-16-136-168.ngrok-free.app/docs
INFO:     154.239.244.250:0 - "OPTIONS /ocr/base64/ HTTP/1.1" 200 OK
INFO:     154.239.244.250:0 - "POST /ocr/base64/ HTTP/1.1" 200 OK
INFO:     154.239.244.250:0 - "POST /ocr/base64/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [6335]
